In [61]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from string import digits
import re
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import re
import os
import io
import time

In [3]:
import os
os.listdir("plant_images")

['.DS_Store', 'test', 'train', 'validation']

In [38]:
!!curl -O http://www.manythings.org/anki/ind-eng.zip
!!unzip ind-eng.zip

['Archive:  ind-eng.zip',
 'replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL',
 '(EOF or read error, treating as "[N]one" ...)']

In [68]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 8471  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "ind.txt"

In [59]:
pd.read_table(data_path , names=['source','target','comments'])

,source,target,comments
0,Run!,Lari!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
1,Who?,Siapa?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Wow!,Wow!,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
3,Help!,Tolong!,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
4,Jump!,Lompat!,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
...,...,...,...
8465,Every student who has graduated from our unive...,Semua mahasiswa yang telah menyelesaikan studi...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
8466,"If you don't want to put on sunscreen, that's ...","Kalau kamu tidak mau pakai tabir surya, ya, te...",CC-BY 2.0 (France) Attribution: tatoeba.org #1...
8467,"When she was finished ironing, Mary switched o...","Ketika dia sudah selesai menyetrika, Mary mema...",CC-BY 2.0 (France) Attribution: tatoeba.org #3...
8468,"Irene Pepperberg, a researcher at Northwestern...","Irene Pepperberg, seorang peneliti di Universi...",CC-BY 2.0 (France) Attribution: tatoeba.org #3...


In [62]:
def preprocess_sentence(sentence):
    
    num_digits= str.maketrans('','', digits)
    
    sentence= sentence.lower()
    sentence= re.sub(" +", " ", sentence)
    sentence= re.sub("'", '', sentence)
    sentence= sentence.translate(num_digits)
    sentence= re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = sentence.rstrip().strip()
    sentence=  'start_ ' + sentence + ' _end'
    
    return sentence

In [64]:
print(preprocess_sentence("Can you do it in thirty minutes?"))

start_ can you do it in thirty minutes ? _end


In [75]:
def create_dataset(path, num_examples):
      
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
  
  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
  return zip(*word_pairs)
sample_size=8470
source, target, comment= create_dataset(data_path, sample_size)

In [57]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
# 
print(max_encoder_seq_length)
# print("Number of samples:", len(input_texts))
# print("Number of unique input tokens:", num_encoder_tokens)
# print("Number of unique output tokens:", num_decoder_tokens)
# print("Max sequence length for inputs:", max_encoder_seq_length)
# print("Max sequence length for outputs:", max_decoder_seq_length)
# 
# input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
# target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
# 
# encoder_input_data = np.zeros(
#     (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
# )
# decoder_input_data = np.zeros(
#     (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
# )
# decoder_target_data = np.zeros(
#     (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
# )
# 
# for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
#     for t, char in enumerate(input_text):
#         encoder_input_data[i, t, input_token_index[char]] = 1.0
#     encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
#     for t, char in enumerate(target_text):
#         # decoder_target_data is ahead of decoder_input_data by one timestep
#         decoder_input_data[i, t, target_token_index[char]] = 1.0
#         if t > 0:
#             # decoder_target_data will be ahead by one timestep
#             # and will not include the start character.
#             decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
#     decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
#     decoder_target_data[i, t:, target_token_index[" "]] = 1.0

163
